In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
basepath = 'drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset'


plain_dir = os.path.join(basepath, 'Training/Plain')
pothole_dir = os.path.join(basepath, 'Training/Rocky')

print("Total training Plain road images: ", len(os.listdir(plain_dir)))
print("Total training Pothole road images: ", len(os.listdir(pothole_dir)))

plain_files = os.listdir(plain_dir)
print(plain_files[:10])

pothole_files = os.listdir(pothole_dir)
print(pothole_files[:10])

Total training Plain road images:  300
Total training Pothole road images:  119
['160.grusvag-gardesgardar-ola-jennersten-ibl-33-902x492.jpg', '16.nhai-eyes-rs-5000-crore-from-roads-bidding.jpg', '83.slider07_wagdhari_ribbanpalli_20_tollway_plaza.JPG', '344.640px-Approaching-Yamuna01_Agra_Lucknow_Expressway_33198644271-1.jpg', '355.BRO-to-use-plastic-roads.jpeg', '8.d336c256e5e7464f8757216009a9be44.jpg', '155.Heavy_traffic_bad_roads_JF_TT.jpg', '137.380-road-Uttarakhand.jpg', '64.1540148417-3697.jpg', '30.highway-401-westbound-collectors-crash.jpg']
['000021.jpg', '000016.jpg', '000007.jpg', '000011.jpg', '000059.jpg', '000058.jpg', '000062.jpg', '000051.jpg', '000063.jpg', '000061.jpg']


In [ ]:
basepath = 'drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset'
plain_dir = os.path.join(basepath, 'Validation/Plain')
pothole_dir = os.path.join(basepath, 'Validation/Rocky')
f = open('sample_test.txt', 'a')
L=[]
for i in os.listdir(pothole_dir):
  if i.endswith(".jpg"):
    f.write("drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset/Validation/Rocky/"+i+"\n")
    L.append("Rocky")
for i in os.listdir(plain_dir):
  if i.endswith(".jpg"):
    f.write("drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset/Validation/Plain/"+i+"\n")
    L.append("plain")

In [ ]:
basepath = 'drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset'
plain_dir = os.path.join(basepath, 'Training/Plain')
pothole_dir = os.path.join(basepath, 'Training/Rocky')

f = open('sample_train.txt', 'a')
for i in os.listdir(pothole_dir):
  if i.endswith(".jpg"):
    f.write("drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset/Training/Rocky/"+i+" Rocky"+"\n")
for i in os.listdir(plain_dir):
  if i.endswith(".jpg"):
    f.write("drive/MyDrive/CSE_575_Road_Condition_Classification_Project/Dataset/Training/Plain/"+i+" plain"+"\n")

In [ ]:
import numpy
from numpy import array
from numpy import mean
from numpy import cov, var
from PIL import Image
from numpy.linalg import eigh, norm
from matplotlib.pyplot import *
import math

# the probability of an image belonging to a class P(picture | class) = P(class, picture) / P(class)
# This means that the probability of the image being in class can be determined by P(x), using a normal distribution
# variance (sigma_sq), mean (avg)

def train_data():
	f = open('sample_train.txt', 'r')
  
	train_images = []

	for line in f:
				line = line.split(' ')
				if line[0].endswith(".jpg"):
						train_images.append((
							(numpy.asarray(Image.open(line[0]).convert('L').resize((64, 64))).flatten()), line[1].split('\n')[0]))
	return train_images

def test_data():

	f = open('sample_test.txt', 'r')
	test_images = []

	for line in f:
					test_images.append(numpy.asarray(Image.open(line.split()[0]).convert('L').resize((64, 64))).flatten())

	return test_images

def train_PCA(data):
	images = []

	for (image, name) in data:
		images.append(image)

	matrix = numpy.asarray(images)
	# print(matrix)

	avg = mean(matrix.T, axis=1)
	center = matrix - avg
	variance = cov(center.T)
	values, vectors = eigh(variance)

	feat_vec = numpy.flip(vectors)[:,:32]
	norm_line = feat_vec.T.dot(center.T)

	return feat_vec, norm_line.T, avg

def eigen_class(eigen, data):

	classed_eigen = dict()

	for index, arr in enumerate(eigen):
		if data[index][1] not in classed_eigen:
			classed_eigen[data[index][1]] = list()
		classed_eigen[data[index][1]].append(arr) 
	
	for key in classed_eigen:
		classed_eigen[key] = numpy.asarray(classed_eigen[key])

	return classed_eigen

def mean_and_var(classed_eigen):
	avg = {}
	vari = {}
	for name in classed_eigen:
		arr = classed_eigen[name]
		# mu = mean(arr.T, axis=1)
		mu = [mean(col) for col in arr.T]
		# cen = arr - mu
		sigma_sq = var(arr.T, axis=1)

		if name not in avg:
			avg[name] = 0
			vari[name] = 0
		avg[name] = mu
		vari[name] = sigma_sq
	return avg, vari


def test_PCA(test_data, training_vectors, avg):
		matrix = numpy.asarray(test_data)
		# print(matrix)
		center = matrix - avg

		test_norm_line = training_vectors.T.dot(center.T)
		
		return test_norm_line.T

def find_distributions(mu, sig_sq, test_line):
	prod = 1
	max_val = -9999
	max_class = list()
	for vec in test_line:
		temp_name = 'X'
		max_val = -9999
		# print(test_line.shape)
		for name in mu:
			prod = 1	
			for index in range(len(vec)): 
				p_x_1 = (2 * 3.14 * sig_sq[name][index]) ** 0.5
				ra = (-(vec[index] - mu[name][index]) ** 2) / (2*sig_sq[name][index])
				p_x_2 = math.exp(ra)
				p_x = p_x_2/p_x_1
				prod *= p_x 
				# print(p_x_1, p_x_2, ra, p_x)
				# print(vec[index])
			if prod > max_val:
				max_val = prod
				temp_name = name
		max_class.append(temp_name)
	return max_class

training_data = train_data()
testing_data = test_data()

print(training_data)
print(testing_data)

vec, line, avg = train_PCA(training_data)

classed_eigen = eigen_class(line, training_data)
mu, sig_sq = mean_and_var(classed_eigen)

test_line = test_PCA(testing_data, vec, avg)
names = find_distributions(mu, sig_sq, test_line)


[(array([252, 252, 252, ...,  48,  35,  21], dtype=uint8), 'Rocky'), (array([ 37,  77, 209, ...,  57,  47,  36], dtype=uint8), 'Rocky'), (array([38, 48, 55, ..., 49, 32, 21], dtype=uint8), 'Rocky'), (array([55, 73, 94, ..., 63, 51, 39], dtype=uint8), 'Rocky'), (array([18, 20, 25, ..., 78, 51, 28], dtype=uint8), 'Rocky'), (array([44, 58, 73, ..., 30, 24, 21], dtype=uint8), 'Rocky'), (array([38, 43, 54, ..., 78, 46, 25], dtype=uint8), 'Rocky'), (array([ 46, 155, 254, ...,  32,  25,  18], dtype=uint8), 'Rocky'), (array([20, 26, 31, ..., 68, 55, 39], dtype=uint8), 'Rocky'), (array([30, 38, 56, ..., 65, 39, 24], dtype=uint8), 'Rocky'), (array([ 76, 105, 135, ...,  34,  33,  29], dtype=uint8), 'Rocky'), (array([26, 32, 37, ..., 67, 49, 37], dtype=uint8), 'Rocky'), (array([21, 25, 31, ..., 56, 49, 37], dtype=uint8), 'Rocky'), (array([100, 101,  94, ...,  27,  18,  15], dtype=uint8), 'Rocky'), (array([45, 54, 47, ..., 70, 76, 57], dtype=uint8), 'Rocky'), (array([31, 42, 59, ..., 59, 46, 41], d

In [ ]:
print(names)
print(L)
accuracy=sum(1 for x,y in zip(names,L) if x == y) / len(L)
accuracy

['Rocky', 'Rocky', 'Rocky', 'plain', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'plain', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'plain', 'plain', 'plain', 'plain', 'plain', 'Rocky', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'Rocky', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'Rocky', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'Rocky', 'plain', 'Rocky', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'Rocky', 'plain', 'plain']
['Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'Rocky', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain', 'plain',

0.8787878787878788